# Laboratorio 1.1
## Calcolare la similarità nelle definizioni dei 4 concetti del dataset



In [70]:
# python -m IPython notebook
from nltk import word_tokenize, SnowballStemmer
from nltk.corpus import stopwords, words
import string
import pandas as pd
import random
import matplotlib.pylab as plt
from operator import itemgetter

snow_stemmer = SnowballStemmer(language='english')

# Crea la variabile stop_words con le stop word e con la punteggiatura
stop_words = set(stopwords.words('english'))
stop_words.update(set(stopwords.words('italian')))
stop_words.update(set(string.punctuation))


# Pre-Processing
## 1. Definizione della funzione per l'estrazione delle informazioni dalla base di dati

In [71]:
media = {}           # Dictionary che conterrà la media aritmetica della lunghezza di ogni definizione per ogni concetto

def extract_data(path='dataset/defs.csv'):
    if path == 'dataset/defs.csv':
        df = pd.read_csv(path, header=0)
        df = df.dropna()
        df.drop(['Partecipante'],axis=1,inplace=True)
        data = df.to_dict()

    elif 'dataset/db.csv':
        df = pd.read_csv(path, header=0, sep=',')
        data = df.values.tolist()

    return data

## 2. Definizione della funzione per la pulitura dell'input
La funzione restituirà un dictionary innestato di questo tipo `{Concetto: {utente 0: [definizione tokenizzata e stemmatizzata]}}`

In [72]:
def sort_indexes(data):
    defs = {}

    for concept in data.keys():
        i = 0
        defs.setdefault(concept,{})
        for index in data[concept]:
            defs[concept].setdefault(i, data[concept][index])
            i += 1
    return defs

In [73]:
def pre_processing(data):
    
    if type(data) == list:
        for element in range(len(data)):
            token = word_tokenize(data[element][0].lower())
            token = snow_stemmer.stem(token[0])
            del data[element]
            data.append(token)

    elif type(data) == dict:

        for names in data.keys():
            for index in data[names]:
                data[names][index] = word_tokenize(data[names][index].lower())
                data[names][index] = list(set([snow_stemmer.stem(word) for word in data[names][index] if word not in stop_words or word != "'s"]) - stop_words)

    return data

## Esecuzione delle funzioni dichiarate

In [74]:
data = extract_data()

# Estrazione dei dati dei concetti in un dictionary
data = sort_indexes(data)

# Pre-processing dell'input
data = pre_processing(data)

# Laboratorio 1.3
## Word Sense Induction
Si contrappone al Word Sense Disambiguation che è un problema computazionale aperto in cui si deve riconoscere il **senso** di una parola in una frase, siccome una parola può essere utilizzata con accezioni differenti.
Il ***Word Sense Induction*** è invece basato su un meccanismo dove si uniscono due parole (creando una pseudo-word) e si da uno score se quando si itera sul corpus si incontra una delle componenti della pseudoword. Lo score può essere positivo se si ci riferisce a una componente e negativo se ci si riferisce alla seconda componente.

In [75]:
# Restituisce una lista con tutti i token distinte utilizzate
def get_tokens(corpus): 
    words = []
    data = pre_processing(corpus)

    if type(corpus) == dict:
        for concept in data.keys():
            for sentence in corpus[concept]:
                for token in corpus[concept][sentence]:
                    if token != "'s":
                        words.append(token)

    elif type(data) == list:
        for element in data:
            if element[0] != "'s":
                words.append(element[0])

    return list(set(words))

In [76]:
print(get_tokens(extract_data()))

['constant', 'human', 'hero', 'obtain', 'clearer', 'person', 'perform', 'student', 'nervous', 'mine', 'block', 'despit', 'easili', 'abil', 'qualiti', 'awe', 'sheet', 'short', 'done', 'drastic', 'persever', 'note', 'restless', 'sharpen', 'emot', 'abnorm', 'multipl', 'compos', 'packag', 'water', 'subject', 'normal', 'cortex', 'equip', 'refin', 'edg', 'scare', 'sever', 'uneas', 'materi', 'usual', 'lightweight', 'inner', 'take', 'caus', 'understand', 'mind', 'allow', 'beyond', 'disturb', 'spiacevol', 'expect', 'handwrit', 'face', 'possibl', 'peopl', 'state', 'lead', 'strength', 'general', 'overcom', 'sharper', 'someon', 'happen', 'inform', 'danger', 'properti', 'withstand', 'bad', 'scari', 'thaht', 'one', 'someth', 'print', 'character', 'blade', 'tool', 'loss', 'negat', 'end', 'rip', 'us', 'make', 'cellulos', 'graphit', 'qualcosa', 'univers', 'creat', 'communic', 'surfac', 'moral', 'sens', 'may', 'avail', 'control', 'action', 'written', 'avoid', 'screen', 'behavior', 'differ', 'consid', 'f

In [77]:
# Restituisce un dictionary con le pseudo-words come chiavi e 0 come valori
def generate_pseudo_words(corpus):
    list_of_tokens = get_tokens(corpus)
    pseudo_words = {}
    scores = {}
    
    for token in range(0, len(list_of_tokens), 2):
        try:
            pseudo_word = list_of_tokens[token] + list_of_tokens[token + 1]
            scores.setdefault(pseudo_word, 0)
            pseudo_words.setdefault(pseudo_word, [list_of_tokens[token], list_of_tokens[token + 1]])

        except IndexError:
            continue
        
    return [scores, pseudo_words]

In [78]:
#print(generate_pseudo_words(data_extraction('dataset/db.csv'))[1])

In [79]:
# Restituisce un nuovo corpus con al posto di alcuni token i token con le pseudo-words, e in più restituisce anche una lista associativa con le pseudo-words e il loro score
def substitute_pseudo_words(corpus, corpus_to_substitute=None):
    corpus = pre_processing(corpus)
    corpus_to_substitute = pre_processing(corpus_to_substitute)
    new_corpus = {}
    pseudo_words_dictionary = generate_pseudo_words(extract_data('dataset/db.csv'))[0]
    pseudo_words_map = generate_pseudo_words(extract_data('dataset/db.csv'))[1]


    if type(corpus) == dict:

        for concept in corpus.keys():
            new_corpus.setdefault(concept, {})
            
            for definition_number in corpus[concept]: 

                if definition_number not in new_corpus[concept].keys():
                    new_corpus[concept].setdefault(definition_number, [])
                    
                    
                for token in corpus[concept][definition_number]:
                    keys = list(pseudo_words_map.keys())
                    combined = '\t'.join(keys)
                    if token in combined:
                        pseudo_word = [pseudo_word for pseudo_word in keys if token in pseudo_word][0]

                        # Se la parola la prima parola che compone la pseudo-word è contenuta nella pseudo-word allora si aggiunge 1
                        if token in pseudo_words_map[pseudo_word][0]:
                            pseudo_words_dictionary[pseudo_word] += 1
                        else: 
                            pseudo_words_dictionary[pseudo_word] -= 1   

                        if pseudo_word != '':
                            token = token.replace(token, pseudo_word)

                    new_corpus[concept][definition_number].append(token)

    
    
    elif type(corpus) == list:
        for element in corpus:
            token = element[0]
            keys = list(pseudo_words_dictionary.keys())
            combined = '\t'.join(keys)
            if token in combined:
                pseudo_word = [pseudo_word for pseudo_word in keys if token in pseudo_word][0]

                # Se la parola la prima parola che compone la pseudo-word è contenuta nella pseudo-word allora si aggiunge 1
                try:
                    if token in pseudo_words_map[pseudo_word][0]:
                        pseudo_words_dictionary[pseudo_word] += 1

                except KeyError:
                    pseudo_words_dictionary.setdefault(pseudo_word, 1)

                try:
                    if token in pseudo_words_map[pseudo_word][1]:
                        pseudo_words_dictionary[pseudo_word] -= 1

                except KeyError:
                    pseudo_words_dictionary.setdefault(pseudo_word, -1)

                if pseudo_word != '':
                    token = token.replace(token, pseudo_word)

            for concept in corpus_to_substitute.keys():
                if concept not in new_corpus.keys():
                    new_corpus.setdefault(concept, {})


                for definition_number in corpus_to_substitute[concept]:
                    new_definition = []

                    # Ogni token viene sostituito con la pseudoword che contiene il token
                    for token in corpus_to_substitute[concept][definition_number]:
                        if token in pseudo_word:
                            #print(f"{token} - {pseudo_word[:len(token)]}")

                            token = token.replace(token, pseudo_word)

                        new_definition.append(token)
                    new_corpus[concept].setdefault(definition_number, new_definition)

    return [new_corpus, pseudo_words_dictionary]           

In [80]:
substitute_pseudo_words(extract_data('dataset/db.csv'), extract_data())

[{'Courage': {0: ['face',
    'situat',
    'fear',
    'properti',
    'allow',
    'ani',
    'feel',
    'despit'],
   1: ['abil', 'someth', 'face', 'fear', 'unpleas', 'us', 'make', 'scar'],
   2: ['abil', 'without', 'face', 'fear', 'thing'],
   4: ['face', 'situat', 'inner', 'particular', 'allow', 'strength', 'thaht'],
   5: ['control', 'abil', 'fear'],
   6: ['abil', 'someth', 'control', 'fear', 'deal', 'unpleas'],
   7: ['abil', 'action', 'avoid', 'take', 'fear', 'riski'],
   8: ['abili', 'without', 'action', 'take', 'fear', 'make', 'choic'],
   9: ['someth', 'abl', 'fear'],
   10: ['abil', 'despit', 'someth', 'frighten'],
   11: ['abil', 'someth', 'peopl', 'scare'],
   13: ['face', 'us', 'situat', 'consid', 'danger', 'allow', 'feel'],
   14: ['scari', 'abil', 'someth', 'may'],
   15: ['drastic', 'abil', 'choic', 'make'],
   16: ['abil', 'overcom', 'fear'],
   17: ['characterist', 'risk', 'person', 'take'],
   18: ['qualiti', 'abl', 'danger', 'scare', 'thing', 'general'],
   19: 

In [81]:
# Score to probability
def score_converter(score, scores_dictionary):
    temp = []
    rate = 0.00

    for value in scores_dictionary.values():
        temp.append(value)
    
    pos_max_score = max(temp)
    neg_max_score = min(temp)

    # Percentuale di quanto lo score è positivo (1.0 se è uguale al massimo)
    if score > 0:
        rate = score / pos_max_score

    elif score < 0:
        # Percentuale di quanto lo score è negativo (1.0 se è uguale al massimo)
        rate = score / neg_max_score
        
    rate = round(rate, 2)
    return rate


In [82]:
score_converter(-10, substitute_pseudo_words(extract_data('dataset/db.csv'), extract_data())[1])

0.45

In [83]:
def wsi(target_corpus, scores, pseudo_words):

    for concept in target_corpus.keys():
        for index in target_corpus[concept]:
            for token_index in range(len(target_corpus[concept][index])):
                token = target_corpus[concept][index][token_index]

                if token in scores.keys():
                    pseudo_word = pseudo_words[token]

                    score = scores[token]

                    if score > 0:
                        target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[0]) 

                    elif score == 0:
                        random_index = random.randint(0, 1)
                        target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[random_index]) 

                    else: 
                        target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[1]) 

    return target_corpus

In [84]:
def wsi2(target_corpus, scores, pseudo_words):

    for concept in target_corpus.keys():
        for index in target_corpus[concept]:
            for token_index in range(len(target_corpus[concept][index])):
                token = target_corpus[concept][index][token_index]

                if token in scores.keys():
                    pseudo_word = pseudo_words[token]

                    score = scores[token]

                    rate = score_converter(score, substitute_pseudo_words(extract_data())[1])
                    rnd = random.randint(0.00, 1.00)

                    if score > 0:
                        if rate > rnd:
                            target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[0]) 
                        else:
                            target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[1]) 

                    elif score == 0:
                        random_index = random.randint(0, 1)
                        target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[random_index]) 

                    else: 
                        if rate > rnd:
                            target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[0]) 
                        else:
                            target_corpus[concept][index][token_index] = token.replace(token, pseudo_word[1]) 

    return target_corpus

In [85]:
def eval_ac(wsi_corpus, real_corpus):
    real_corpus = pre_processing(real_corpus)
    right_previsions = 0
    total = 0
    for concept in real_corpus.keys():
        for index in real_corpus[concept]:
            for token_index in range(len(real_corpus[concept][index])):
                token = real_corpus[concept][index][token_index]
                
                if token == wsi_corpus[concept][index][token_index]:
                    right_previsions += 1
                    
                total += 1
    accuracy = right_previsions / total

    return accuracy

In [86]:
# Main
input_corpus = substitute_pseudo_words(extract_data())[0]

scores = substitute_pseudo_words(extract_data('dataset/db.csv'), extract_data())[1]

pseudo_words = generate_pseudo_words(extract_data('dataset/db.csv'))[1]

wsi_corpus = wsi2(input_corpus, scores, pseudo_words)

accuracy = eval_ac(wsi_corpus, extract_data())

In [87]:
print(round(accuracy, 2))

0.76
